In [3]:
pip install google-cloud-compute google-cloud-monitoring

   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.4 MB 1.4 MB/s eta 0:00:03
    --------------------------------------- 0.1/3.4 MB 1.7 MB/s eta 0:00:03
    --------------------------------------- 0.1/3.4 MB 1.7 MB/s eta 0:00:03
   -- ------------------------------------- 0.2/3.4 MB 1.4 MB/s eta 0:00:03
   --- ------------------------------------ 0.3/3.4 MB 1.7 MB/s eta 0:00:02
   ---------- ----------------------------- 0.9/3.4 MB 3.4 MB/s eta 0:00:01
   ---------- ----------------------------- 0.9/3.4 MB 3.4 MB/s eta 0:00:01
   -------------- ------------------------- 1.2/3.4 MB 3.6 MB/s eta 0:00:01
   --------------- ------------------------ 1.3/3.4 MB 3.3 MB/s eta 0:00:01
   -------------------- ------------------- 1.8/3.4 MB 4.0 MB/s eta 0:00:01
   -------------------- ------------------- 1.8/3.4 MB 4.0 MB/s eta 0:00:01
   ------------------------- -------------- 2.2/3.4 MB 4.0 MB/s eta 0:00:01
   ----------------

In [1]:
import csv
from datetime import datetime, timedelta,date
from google.cloud import monitoring_v3
from google.cloud import monitoring_v3
from google.oauth2 import service_account
from google.cloud import compute_v1

# Config
PROJECT_ID = "glassy-song-469315-m7"  
# Client
creds = service_account.Credentials.from_service_account_file(
    r"D:/JAMES SINGH/Mtech/Semester 8/Dataset/Program/key.json"
)
client = monitoring_v3.MetricServiceClient(credentials=creds)
# Metrics parameters
metrics_list = {
    "Percentage CPU (Average)": {
        "metric": "compute.googleapis.com/instance/cpu/utilization",
        "aggregation": "mean",
        "multiplier": 100, },
    "Disk Read Bytes (Sum)": {
        "metric": "compute.googleapis.com/instance/disk/read_bytes_count",
        "aggregation": "sum",},
    "Disk Write Bytes (Sum)": {
        "metric": "compute.googleapis.com/instance/disk/write_bytes_count",
        "aggregation": "sum", },
    "Network In Total (Sum)": {
        "metric": "compute.googleapis.com/instance/network/received_bytes_count",
        "aggregation": "sum",},
    "Network Out Total (Sum)": {
        "metric": "compute.googleapis.com/instance/network/sent_bytes_count",
        "aggregation": "sum",
    },
}


# Get metric for one day
def get_metric(instance_id, metric_type, aggregation, start_time, end_time, multiplier=1):
    interval = monitoring_v3.TimeInterval(
        start_time=start_time,
        end_time=end_time,
    )
    aggregation_obj = monitoring_v3.Aggregation(
        alignment_period={"seconds": 86400},  # 1 day
        per_series_aligner=(
            monitoring_v3.Aggregation.Aligner.ALIGN_MEAN
            if aggregation == "mean"
            else monitoring_v3.Aggregation.Aligner.ALIGN_SUM
        ),
        cross_series_reducer=monitoring_v3.Aggregation.Reducer.REDUCE_NONE,
    )

    results = client.list_time_series(
        request={
            "name": f"projects/{PROJECT_ID}",
            "filter": f'metric.type="{metric_type}" AND resource.type="gce_instance" AND resource.labels.instance_id="{instance_id}"',
            "interval": interval,
            "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
            "aggregation": aggregation_obj,
        }
    )

    for ts in results:
        for point in ts.points:
            if point.value.double_value:
                return point.value.double_value * multiplier
            elif point.value.int64_value:
                return point.value.int64_value * multiplier
    return 0


compute_client = compute_v1.InstancesClient(credentials=creds)
instances = []
agg_req = compute_v1.AggregatedListInstancesRequest(project=PROJECT_ID)
for zone, resp in compute_client.aggregated_list(request=agg_req):
    if resp.instances:
        for inst in resp.instances:
            instances.append((inst.id, inst.name))


# Collect data row-wise
rows = []
end_time = datetime.utcnow()
START_DATE=date(2025, 8, 16)
RUN_DATE=date(2025, 10, 2)
DAYS=(RUN_DATE-START_DATE).days+1

for i in range(DAYS):  # Number of days
    day_end = (end_time - timedelta(days=i)).replace(hour=23, minute=59, second=59, microsecond=0)
    day_start = day_end.replace(hour=0, minute=0, second=0, microsecond=0)
    date_str = day_start.strftime("%d-%m-%Y")

    interval = monitoring_v3.TimeInterval(
        start_time={"seconds": int(day_start.timestamp())},
        end_time={"seconds": int(day_end.timestamp())},
    )

    for instance_id, instance_name in instances:
        row = {
            "Date": date_str,
            "InstanceName": instance_name,
        }

        for col, metric in metrics_list.items():
            value = get_metric(
                instance_id,
                metric["metric"],
                metric["aggregation"],
                interval.start_time,
                interval.end_time,
                metric.get("multiplier", 1),
            )
            row[col] = value

        rows.append(row)

# Output file
output_file = "gcp_vm_metrics_daywise.csv"
columns = ["Date", "InstanceName"] + list(metrics_list.keys())

with open(output_file, mode="w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=columns)
    writer.writeheader()
    writer.writerows(rows)

print(f"GCP Metrics exported to {output_file}")


C:\Users\JAMES SINGH\AppData\Local\Temp\ipykernel_15208\4092148789.py:83: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_time = datetime.utcnow()


GCP Metrics exported to gcp_vm_metrics_daywise.csv
